In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm import tqdm
import pickle
from copy import deepcopy
import uuid
from urllib.request import urlretrieve
from zipfile import ZipFile

In [2]:
"""
Uncomment the below code if you are using google colab.
"""

!git clone https://github.com/shubhamgantayat/Traffic-Signs-Detection-Testing.git
%cd Traffic-Signs-Detection-Testing/notebook/

Cloning into 'Traffic-Signs-Detection-Testing'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 64 (delta 13), reused 24 (delta 11), pack-reused 34
Unpacking objects: 100% (64/64), done.
/content/Traffic-Signs-Detection-Testing/notebook


In [3]:
ZIP_FILE_PATH = Path("../raw_data/data.zip")
FILE_PATH = Path("../data/")
if not os.path.exists(FILE_PATH):
    os.makedirs(FILE_PATH, exist_ok=True)
    with ZipFile(ZIP_FILE_PATH, "r") as f:
        f.extractall(FILE_PATH)

In [4]:
labels = pd.read_csv("../data/labels_YOLO_Traffic_Sign.csv")

In [5]:
labels

,ClassID,Description
0,1,Pedestrian
1,2,Left Sharp Curve
2,3,Right Sharp Curve
3,4,No Over Taking
4,5,Turn Left
5,6,Turn Right
6,7,Speed Limit 70
7,8,Speed Limit 50
8,9,Speed Limit 30
9,10,Speed Limit 20


In [6]:
nc = len(labels)

In [7]:
classes = str(list(labels["Description"]))

In [8]:
desc = f"""train: ../data/TrafficSignLocalizationandDetection/train/images
val: ../data/TrafficSignLocalizationandDetection/valid/images
test: ../data/TrafficSignLocalizationandDetection/test/images

nc: {nc}
names: {classes}
"""

with open("../data/data.yaml", "w") as f:
    f.write(desc)

# Setting up yolov5

In [9]:
%cd ../

/content/Traffic-Signs-Detection-Testing


In [10]:
!git clone https://github.com/ultralytics/yolov5.git  # clone repo
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 13176, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 13176 (delta 6), reused 12 (delta 4), pack-reused 13156
Receiving objects: 100% (13176/13176), 11.97 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (9156/9156), done.
/content/Traffic-Signs-Detection-Testing/yolov5


In [11]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 596 kB 3.2 MB/s 
Setup complete. Using torch 1.11.0+cu113 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


In [12]:
%cat ../data/data.yaml

train: ../data/TrafficSignLocalizationandDetection/train/images
val: ../data/TrafficSignLocalizationandDetection/valid/images
test: ../data/TrafficSignLocalizationandDetection/test/images

nc: 39
names: ['Pedestrian', 'Left Sharp Curve', 'Right Sharp Curve', 'No Over Taking', 'Turn Left', 'Turn Right', 'Speed Limit 70', 'Speed Limit 50', 'Speed Limit 30', 'Speed Limit 20', 'Speed Limit 60', 'No Waiting', 'Road Work', 'TrafficSign', 'Snow Warning Sign', 'Slippery Road', 'Give Way', 'RoundAbout', 'No Stopping', 'Go Straight', 'Danger Ahead', 'Go Right or Straight', 'Go Left or Straight', 'End of Right Road (Go straight)', 'Stop', 'Right Curve Ahead', 'Left Curve Ahead', 'Cycle Zone', 'Speed Limit 80', 'Speed Limit 120', 'No Over Taking Trucks', 'Truck Sign', 'No Entry', 'Traffic Signals Ahead', 'Speed Limit 100', 'End of Left Road (Go straight)', 'Deer Zone', 'Huddle Road', 'No Truck Sign']


In [13]:
import yaml
with open("../data/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [14]:
num_classes

'39'

In [15]:
%cat /content/Traffic-Signs-Detection-Testing/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [16]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [17]:
%%writetemplate /content/Traffic-Signs-Detection-Testing/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [22]:
%%time
%cd /content/Traffic-Signs-Detection-Testing/yolov5/
!python train.py --img 416 --batch 16 --epochs 50 --data '../data/data.yaml' --cfg ./models/custom_yolov5s.yaml --weights runs/train/yolov5s_results/weights/best.pt --name yolov5s_results

/content/Traffic-Signs-Detection-Testing/yolov5
train: weights=runs/train/yolov5s_results/weights/best.pt, cfg=./models/custom_yolov5s.yaml, data=../data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-174-gc4cb7c6 torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cl

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
%cp /content/Traffic-Signs-Detection-Testing/yolov5/runs/train/yolov5s_results2/weights/best.pt /content/gdrive/MyDrive/My\ Projects/best2.pt